## This notebook was used to process cycling infrastructure data from the City of Atlanta and the Atlanta Regional Commission
---
**NOTE: code will not run without the files**

In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

from bikewaysim.paths import config, root

In [2]:
studyarea_geo = gpd.read_file(config['studyarea_fp']).to_crs('epsg:4326')

## City of Atlanta 
Has year installed date and other notes. Contains sharrows. Also contains "planned" infrastructure that may not have actually been completed. Some street names have changed like "Confederate" to "United". Data were retrieved from:
- https://services2.arcgis.com/zLeajbicrDRLQcny/ArcGIS/rest/services/Bike_Facilities_Public_View/FeatureServer/0

In [3]:
#TODO, resolve difference between the old inventory and the old as some streets had their facilities upgraded recently (Edgewood) and (Confederate/United)
coa = gpd.read_file(config['coa_bike_fp'],mask=studyarea_geo)
coa.to_crs(config['projected_crs_epsg'],inplace=True)

# only keep existing and funded ones where the year is defined
coa = coa[(coa['Status'].isin(['Existing'])) & coa['YearInstalled'].notna()]

# remove uncessary columns
coa.drop(columns=['GlobalID','Shape__Length','LengthMi','CrossSectionNotes','Status'],inplace=True)

#rename the id/year column
coa.rename(columns={'OBJECTID':'id','YearInstalled':'year'},inplace=True)

#mark facilities that need to be dated
coa['need_date'] = (coa['year'] >= 2012) & (coa['year'] <= 2016)

# rename columns for consistency
coa.columns = ['coa_'+col.lower() if col != 'geometry' else 'geometry' for col in coa.columns.tolist()]

# convert facility type to OSM (use highest protection if two different types)
osm_types = ['sharrow','bike lane','buffered bike lane','cycletrack','multi use path']
coa_conversion = {
       'Protected Bike Lane': osm_types[3], 
       'Protected Bike Lane / Bike Lane': osm_types[3],
       'Two-Way Cycle Track': osm_types[3], 
       'Uphill Bike Lane / Downhill Sharrows': osm_types[1],
       'Sharrows': osm_types[0], 
       'Bike Lane': osm_types[1], 
       'Bike Lane ': osm_types[1],
       'Uphill Buffered Bike Lane / Downhill Sharrows': osm_types[2],
       'Buffered Bike Lane': osm_types[2], 
       'Buffered Contraflow Bike Lane / Bike Lane': osm_types[1],
       'Shared-Use Path': osm_types[4], 
       'Neighborhood Greenway': osm_types[0], 
       'Bike Lane / Sharrows': osm_types[1],
       'Shared-Use Path / Bike Lane': osm_types[4], 
       'Buffered Bike Lane / Bike Lane': osm_types[2],
       'Buffered Bike Lane / Shared-Use Path': osm_types[4],
       'Shared-Use Path / Sharrows': osm_types[4],
       'Uphill Protected Bike Lane / Downhill Sharrows': osm_types[3], 
       'Shared Path': osm_types[4]
}
coa['coa_osm_type'] = coa['coa_facilitytype'].map(coa_conversion)

# drop sharrow only facilities
coa = coa[coa['coa_osm_type']!=osm_types[0]]

#export
coa.to_file(config['bicycle_facilities_fp']/'reference_layers.gpkg',layer='coa')

## Atlanta Regional Commission
Retrieved from https://opendata.atlantaregional.com/search?q=bicycle%20inventory

In [4]:
arc = gpd.read_file(config['arc_bike_fp'],mask=studyarea_geo)
arc.to_crs(config['projected_crs_epsg'],inplace=True)

# the only na values are OTP
#arc[arc['Year_2'].isna()].explore()

# remove post 2016
# arc = arc[arc['Year_2'] < 2016]

#clean the columns
arc = arc[['OBJECTID_1','Name', 'spec','Width', 'Material', 'Year_2', 'geometry']]

#rename the id column
arc.rename(columns={'OBJECTID_1':'id','Year_2':'year'},inplace=True)

#mark facilities that need to be dated
arc['need_date'] = (arc['year'] >= 2012) & (arc['year'] <= 2016)

# rename columns for consistency
arc.columns = ['arc_'+col.lower() if col != 'geometry' else 'geometry' for col in arc.columns.tolist()]

# remove these facilities
remove = ['Paved shoulder','Park Trail','Park Path','Campus Path']
arc = arc[arc['arc_spec'].isin(remove)==False]

# convert to osm name
osm_types = ['sharrow','bike lane','buffered bike lane','cycletrack','multi use path']
arc_conversion = {
    'Hard surface multi-use path':osm_types[4],
    'Protected bike lane':osm_types[3],
    'Two way cycle track':osm_types[3],
    'Uphill bike lanes / downhill sharrows':osm_types[1],
    'Conventional bike lane':osm_types[1],
    'Bike lane':osm_types[1],
    'Bike lane/w parking':osm_types[1],
    'Buffered bike lane':osm_types[2],
    'Bike lane with parking':osm_types[1],
    'Buffered bike lane (BUS ONLY lane)':osm_types[2],
    # 'Paved shoulder':osm_types[1], #drop these
    'Buffered/bike lane':osm_types[2],
    'Shared use path or greenway':osm_types[4],
    'Buffered bike lane on north side':osm_types[2],
    'Uphill bike lane / dowhill sharrow':osm_types[1],
    'Two way cycletrack':osm_types[3],
    'Side path':osm_types[4],
    'Protected cycle track':osm_types[3],
    'Two way raised cycletrack':osm_types[3],
    'Raised cycle track':osm_types[3],
    # 'Closed bridge':osm_types[4],
    'Bike lane and Side path':osm_types[1],
    'Uphill bike lane/downhill sharrow':osm_types[1],
    # 'Parking lot connection':osm_types[4],
    'Side Path':osm_types[4],
    'Climbing lane':osm_types[1],
    ' Hard surface multi-use path':osm_types[4],
    'Cycle Track':osm_types[3],
    'Sidepath':osm_types[4],
    'Uphill bike land/downhill sharrow':osm_types[1],
    'Uphill bike lane / downhill sharrow':osm_types[1],
    'Park Trail':osm_types[4],
    'Bike Lane with parking':osm_types[1],
    'Buffered Bike Lane':osm_types[2],
    'Protected Bike Lane':osm_types[3],
    'Campus Path':osm_types[4],
    'On-street 2-way Cycle Track':osm_types[3],
    'Bike Lane':osm_types[1],
    'Park Path':osm_types[4],
    'shared path or greenway':osm_types[4],
    'side path':osm_types[4] 
}
arc['arc_osm_type'] = arc['arc_spec'].map(arc_conversion)
arc = arc[arc['arc_osm_type'].notna()] # drop any unidentified features

# drop these specific features
drop = [503]
arc = arc[arc['arc_id'].isin(drop)==False]

#export
arc.to_file(config['bicycle_facilities_fp']/'reference_layers.gpkg',layer='arc')